In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import my_txtutils
import re
import nltk

In [13]:
mypath='./new_txts/'
txts='./txts/'
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(txts) if isfile(join(txts, f))]
print(onlyfiles)

['acts_new.txt', 'gal_eph_new.txt', 'heb_new.txt', 'jam_jud_new.txt', 'john_new.txt', 'jud_rev_new.txt', 'luke_8_john_new.txt', 'mark01_new.txt', 'matt02_new.txt', 'matt_new.txt', 'phil_col_new.txt', 'thes_tim_new.txt', 'tit_phl_new.txt']


In [261]:
#large files
def large_f():
    #filenames = ['file1.txt', 'file2.txt', ...]
    with open('path/to/output/file', 'w') as outfile:
        for fname in onlyfiles:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)

In [22]:
#small files
def small_f():
    #filenames = ['file1.txt', 'file2.txt', ...]
    with open(mypath+'full.txt', 'w') as outfile:
        for fname in onlyfiles:
            print('writing file',fname)
            with open(txts+fname) as infile:
                outfile.write(infile.read())
    return open(mypath+"full.txt").read()

In [24]:
txt=small_f()
words=re.sub(' +', ' ',txt.replace('\n',' ')).split(' ')
len(words)

writing file acts_new.txt
writing file gal_eph_new.txt
writing file heb_new.txt
writing file jam_jud_new.txt
writing file john_new.txt
writing file jud_rev_new.txt
writing file luke_8_john_new.txt
writing file mark01_new.txt
writing file matt02_new.txt
writing file matt_new.txt
writing file phil_col_new.txt
writing file thes_tim_new.txt
writing file tit_phl_new.txt


649190

In [25]:
len(txt)

6507438

In [33]:
# these must match what was saved !
ALPHASIZE = my_txtutils.ALPHASIZE
NLAYERS = 3
INTERNALSIZE = 512

ok02 ="checkpoints/rnn_train_1560839489-30000000" # okrika 2019-06-18
ok03 ="checkpoints/rnn_train_1563601394-60000000" # okrika 2019-07-20

author=ok03

ncnt = 0
with tf.Session() as sess:
    # new_saver = tf.train.import_meta_graph('checkpoints/rnn_train_1512567262-0.meta')
    new_saver = tf.train.import_meta_graph(author+'.meta')
    new_saver.restore(sess, author)
    x = my_txtutils.convert_from_alphabet(ord("L"))
    x = np.array([[x]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1

    # initial values
    y = x
    h = np.zeros([1, INTERNALSIZE * NLAYERS], dtype=np.float32)  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]
    for i in range(1000):
        yo, h = sess.run(['Yo:0', 'H:0'], feed_dict={'X:0': y, 'pkeep:0': 1., 'Hin:0': h, 'batchsize:0': 1})
        #print(tf.shape(yo))
        #print(len(yo))
        c = my_txtutils.sample_from_probabilities(yo, topn=2)
        #y = np.array([[c]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1
        c = chr(my_txtutils.convert_to_alphabet(c))
        print(c, end="")
        if c == '\n':
            ncnt = 0
        else:
            ncnt += 1
        if ncnt == 100:
            print("")
            ncnt = 0


INFO:tensorflow:Restoring parameters from checkpoints/rnn_train_1563601394-60000000
 i   i   i   i i           i i       i     i       i         i i i i     i i         i   i i       i
         i i i i   i     i         i   i i   i i i       i i     i i i   i i         i i     i i i  
   i i i i     i   i i       i i       i   i i i i   i i           i   i i       i   i i   i i     i
 i   i           i       i                 i i   i i     i i   i   i   i     i i   i   i i i       i
 i   i   i             i     i     i   i   a   a a a a a     a a a a   a     a   a a     a   a   a a
   a     a         a   a           a a a a     a   a a           a a a     a a         a   a a      
 a a     a a                 a         a a   a a a   a a   a a   a a a a     a a   a   a           a
         a a a   a   a a a             a a a a a a a     a a     a a a     a   a     a           a  
 a       a   a a     a a   a a a   a   a a a     a   a       a         a a         a   a a   a     a
       

In [67]:
def pcx(txt):
    # these must match what was saved !
    ALPHASIZE = my_txtutils.ALPHASIZE
    NLAYERS = 3
    INTERNALSIZE = 512

    ok02 ="checkpoints/rnn_train_1560839489-30000000" # okrika 2019-06-18
    ok03 ="checkpoints/rnn_train_1563601394-60000000" # okrika 2019-07-20

    author=ok03

    ncnt = 0
    with tf.Session() as sess:
        # new_saver = tf.train.import_meta_graph('checkpoints/rnn_train_1512567262-0.meta')
        new_saver = tf.train.import_meta_graph(author+'.meta')
        new_saver.restore(sess, author)
        x = my_txtutils.convert_from_alphabet(ord(txt[0]))
        x = np.array([[x]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1

        # initial values
        y = x
        h = np.zeros([1, INTERNALSIZE * NLAYERS], dtype=np.float32)  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]
        arr=[]
        for i in range(len(txt)-1):
            yo, h = sess.run(['Yo:0', 'H:0'], feed_dict={'X:0': y, 'pkeep:0': 1., 'Hin:0': h, 'batchsize:0': 1})
            c = my_txtutils.convert_from_alphabet(ord(txt[i+1]))
            y = np.array([[c]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1
            y1=np.reshape(yo,[ALPHASIZE,-1])
            y1=y1.flatten()
            #c = chr(my_txtutils.convert_to_alphabet(i))
            print(txt[i],':',c,':',y1[c],',max=',np.max(y1),'=',chr(my_txtutils.convert_to_alphabet(np.argmax(y1))))
            arr.append(y1[c])
    return np.prod(arr)**(-1/len(txt))

In [68]:
pcx('hello world')

INFO:tensorflow:Restoring parameters from checkpoints/rnn_train_1563601394-60000000
h : 71 : 0.017361918 ,max= 0.2139146 =  
e : 78 : 0.008416108 ,max= 0.24278364 = a
l : 78 : 0.00042135394 ,max= 0.9912458 =  
l : 81 : 0.07286589 ,max= 0.59736776 = e
o : 2 : 2.351578e-06 ,max= 0.99999464 =  
  : 89 : 0.006226908 ,max= 0.5184772 =  
w : 81 : 1.9324569e-08 ,max= 0.9999999 =  
o : 84 : 0.0021418824 ,max= 0.604625 = a
r : 78 : 8.288168e-09 ,max= 0.99999976 =  
l : 70 : 0.00027098384 ,max= 0.48159385 = i


2978.628928604615

In [59]:
# these must match what was saved !
ALPHASIZE = my_txtutils.ALPHASIZE
NLAYERS = 3
INTERNALSIZE = 512

ok02 ="checkpoints/rnn_train_1560839489-30000000" # okrika 2019-06-18
ok03 ="checkpoints/rnn_train_1563601394-60000000" # okrika 2019-07-20

author=ok03

ncnt = 0
with tf.Session() as sess:
    # new_saver = tf.train.import_meta_graph('checkpoints/rnn_train_1512567262-0.meta')
    new_saver = tf.train.import_meta_graph(author+'.meta')
    new_saver.restore(sess, author)
    x = my_txtutils.convert_from_alphabet(ord("L"))
    x = np.array([[x]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1

    # initial values
    y = x
    h = np.zeros([1, INTERNALSIZE * NLAYERS], dtype=np.float32)  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]
    yo, h = sess.run(['Yo:0', 'H:0'], feed_dict={'X:0': y, 'pkeep:0': 1., 'Hin:0': h, 'batchsize:0': 1})
    y=np.reshape(yo,[98,-1])
    y=y.flatten()
    v="hello world"
    for i in range(ALPHASIZE):
        #print(tf.shape(yo))
        #print(len(yo))
        #c = my_txtutils.sample_from_probabilities(yo, topn=2)
        #y = np.array([[c]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1
        c = chr(my_txtutils.convert_to_alphabet(i))
        print(c,':',y[i])
        if c == '\n':
            ncnt = 0
        else:
            ncnt += 1
        if ncnt == 100:
            print("")
            ncnt = 0

INFO:tensorflow:Restoring parameters from checkpoints/rnn_train_1563601394-60000000
  : 0.02731734
	 : 0.009119334
  : 0.029877855
! : 0.007832738
" : 0.00781383
# : 0.007916751
$ : 0.007920723
% : 0.007974687
& : 0.008838342
' : 0.008898315
( : 0.009162439
) : 0.009861022
* : 0.008206715
+ : 0.008296969
, : 0.011283324
- : 0.010530039
. : 0.010989632
/ : 0.008403456
0 : 0.007671907
1 : 0.008130989
2 : 0.008053313
3 : 0.008033515
4 : 0.007931977
5 : 0.008146205
6 : 0.008111852
7 : 0.007924892
8 : 0.00794537
9 : 0.008214056
: : 0.008255819
; : 0.008002921
< : 0.0078708455
= : 0.007675543
> : 0.008108395
? : 0.0079168985
@ : 0.008103323
A : 0.007865107
B : 0.008132115
C : 0.008002383
D : 0.0077815484
E : 0.007962899
F : 0.007916055
G : 0.008172119
H : 0.008301339
I : 0.008191501
J : 0.008042702
K : 0.008129442
L : 0.007832516
M : 0.008023312
N : 0.008021216
O : 0.008252282
P : 0.0078577
Q : 0.00826596
R : 0.007967731
S : 0.007995544
T : 0.008093961
U : 0.008012479
V : 0.0079518845
W : 0.

In [66]:
2**2

4

In [46]:
my_txtutils.convert_from_alphabet(ord('i'))

75